## 1. 입력한 질문 벡터화 및 차원축소 

In [1]:
from sentence_transformers import SentenceTransformer

# KoSentence-BERT 모델 불러오기
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [2]:
question = '오른쪽 머리에서 찌릿하는 두통은 뭔가요?'
question_vec = model.encode([question], batchsize=32)

In [3]:
question_vec.shape

(1, 768)

In [4]:
import joblib

# 저장된 RandomForestClassifier 모델 불러오기
rf_model = joblib.load('./headache_randomforest.pkl')
scaler = joblib.load('./headache_scaler.pkl')
pca = joblib.load('./headache_pca.pkl')

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from versio

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

question_ss = scaler.transform(question_vec)
question_pca = pca.transform(question_ss)

In [6]:
# 예측
result = rf_model.predict(question_pca)
result

array([3], dtype=int64)

In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 질문 데이터 불러오기
question_df = pd.read_csv('두통 질문.csv', index_col=0)

le = LabelEncoder()
le_intention = le.fit_transform(question_df['intention'])
le_intention


array([0, 0, 0, ..., 5, 5, 5])

In [8]:
label = le.inverse_transform([result])
label

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array(['증상'], dtype=object)

## 2. faiss
- 대규모 데이터셋에서 빠르고 효율적으로 문장 유사도 검색.
- faiss로 벡터 DB 구축
- 사용자가 입력한 질문을 벡터화하고 faiss를 통해 가장 유사한 벡터 검색

### 답변 임베딩
#### 1) 텍스트 나누기
- 각 언어 모델은 한 번에 처리할 수 있는 텍스트의 길이(토큰수)가 제한적이다.
- 한번에 처리하는 텍스트 길이에 제한을 두어 큰 텍스트를 처리할 때 효율적으로 하기 위함.

In [9]:
import faiss
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [22]:
test_sentence = '두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 자세한 관리는 두통 예방에 도움이 될 수 있습니다. 컴퓨터를 사용할 때는 모니터를 눈높이에 배치하는 것이 좋습니다. 의자에 앉을 때는 등을 펴고 허리를 곧게 세워 앉습니다. 책을 읽을 때는 눈높이에 맞추어 읽으며 책을 읽을 때는 한손으로 받친 자세로 읽는 것이 좋습니다. 모니터를 멀리 배치할 경우 눈의 피로를 줄일 수 있습니다. 긴 시간동안 앉아 있는 경우에는 일어나서 스트레칭을 해주거나 몸을 좌우로 비틀어서 자세를 풀어주는 것이 좋습니다. 또한, 일상생활에서 적절한 자세를 유지하는 것과 충분한 수면을 취함으로써 두통 예방에 도움을 줄 수 있습니다. 올바른 자세와 수면, 스트레칭을 통해 일상 생활에서 올바른 자세를 유지하는 것이 두통 예방에 도움이 됩니다.'
re.split(r'(?<=[.!?]) +', test_sentence)

['두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다.',
 '일상 생활에서의 자세한 관리는 두통 예방에 도움이 될 수 있습니다.',
 '컴퓨터를 사용할 때는 모니터를 눈높이에 배치하는 것이 좋습니다.',
 '의자에 앉을 때는 등을 펴고 허리를 곧게 세워 앉습니다.',
 '책을 읽을 때는 눈높이에 맞추어 읽으며 책을 읽을 때는 한손으로 받친 자세로 읽는 것이 좋습니다.',
 '모니터를 멀리 배치할 경우 눈의 피로를 줄일 수 있습니다.',
 '긴 시간동안 앉아 있는 경우에는 일어나서 스트레칭을 해주거나 몸을 좌우로 비틀어서 자세를 풀어주는 것이 좋습니다.',
 '또한, 일상생활에서 적절한 자세를 유지하는 것과 충분한 수면을 취함으로써 두통 예방에 도움을 줄 수 있습니다.',
 '올바른 자세와 수면, 스트레칭을 통해 일상 생활에서 올바른 자세를 유지하는 것이 두통 예방에 도움이 됩니다.']

In [23]:
# 2문장씩 문장을 나누는 함수
def split_sentences(sentences, max_sentences=2):
    sentence_list = re.split(r'(?<=[.!?]) +', sentences) # 문장을 마침표, 느낌표, 물음표 뒤에오는 공백을 기준으로 나눔(공백 뒤에 +를 붙여서 하나이상을 의미)
    chunks = []
    for i in range(0, len(sentence_list), max_sentences): # max_sentences 간격만큼
        chunks.append(' '.join(sentence_list[i:i + max_sentences])) # join으로 두개의 문장 공백으로 연결, 리스트 대신 문자열로 생성됨.
    
    return chunks


In [24]:
split_sentences(test_sentence)

['두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 자세한 관리는 두통 예방에 도움이 될 수 있습니다.',
 '컴퓨터를 사용할 때는 모니터를 눈높이에 배치하는 것이 좋습니다. 의자에 앉을 때는 등을 펴고 허리를 곧게 세워 앉습니다.',
 '책을 읽을 때는 눈높이에 맞추어 읽으며 책을 읽을 때는 한손으로 받친 자세로 읽는 것이 좋습니다. 모니터를 멀리 배치할 경우 눈의 피로를 줄일 수 있습니다.',
 '긴 시간동안 앉아 있는 경우에는 일어나서 스트레칭을 해주거나 몸을 좌우로 비틀어서 자세를 풀어주는 것이 좋습니다. 또한, 일상생활에서 적절한 자세를 유지하는 것과 충분한 수면을 취함으로써 두통 예방에 도움을 줄 수 있습니다.',
 '올바른 자세와 수면, 스트레칭을 통해 일상 생활에서 올바른 자세를 유지하는 것이 두통 예방에 도움이 됩니다.']

In [25]:
# chunks 속 2문장씩 결합되어있는 문장들의 길이가 너무 긴 경우 나눠주기 위함.
def split_text_with_recursive_splitter(text, chunk_size=300, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size, # 최대 몇자로 설정할지
        chunk_overlap = chunk_overlap # 50개 겹쳐서 시작
    )

    return text_splitter.split_text(text)

In [26]:
split_text_with_recursive_splitter(test_sentence)

['두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 자세한 관리는 두통 예방에 도움이 될 수 있습니다. 컴퓨터를 사용할 때는 모니터를 눈높이에 배치하는 것이 좋습니다. 의자에 앉을 때는 등을 펴고 허리를 곧게 세워 앉습니다. 책을 읽을 때는 눈높이에 맞추어 읽으며 책을 읽을 때는 한손으로 받친 자세로 읽는 것이 좋습니다. 모니터를 멀리 배치할 경우 눈의 피로를 줄일 수 있습니다. 긴 시간동안 앉아 있는 경우에는 일어나서 스트레칭을 해주거나 몸을 좌우로 비틀어서 자세를 풀어주는 것이 좋습니다. 또한,',
 '일어나서 스트레칭을 해주거나 몸을 좌우로 비틀어서 자세를 풀어주는 것이 좋습니다. 또한, 일상생활에서 적절한 자세를 유지하는 것과 충분한 수면을 취함으로써 두통 예방에 도움을 줄 수 있습니다. 올바른 자세와 수면, 스트레칭을 통해 일상 생활에서 올바른 자세를 유지하는 것이 두통 예방에 도움이 됩니다.']

In [16]:
# # 질문 불러오기
# question_df = pd.read_csv('두통 질문.csv', index_col=0)
# question_df

In [27]:
# 답변 불러오기
answer_df = pd.read_csv('두통 답변.csv', index_col=0)
answer_df

,disease_category,disease_name,intention,answer_intro_conclusion,answer
0,응급질환,두통,예방,두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 ...,두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 ...
1,응급질환,두통,예방,"긴장성 두통을 예방하기 위한 몇 가지 조치를 취할 수 있습니다. 마지막으로, 의사나...","긴장성 두통을 예방하기 위한 몇 가지 조치를 취할 수 있습니다. 먼저, 건강한 생활..."
2,응급질환,두통,예방,두통 예방을 위한 생활습관은 다양한 방법으로 개선할 수 있습니다. 일상적인 습관 중...,두통 예방을 위한 생활습관은 다양한 방법으로 개선할 수 있습니다. 일상적인 습관 중...
3,응급질환,두통,예방,머리 통증을 예방하기 위해 몇 가지 전략을 시도해볼 수 있습니다. 이러한 전략들은 ...,머리 통증을 예방하기 위해 몇 가지 전략을 시도해볼 수 있습니다. 스트레스를 관리하...
4,응급질환,두통,예방,스트레스는 긴장성 두통을 유발하는 중요한 요인 중 하나입니다. 스트레스를 줄이는 것...,스트레스는 긴장성 두통을 유발하는 중요한 요인 중 하나입니다. 스트레스를 줄이는 것...
...,...,...,...,...,...
5513,응급질환,두통,치료,"두통은 우리 일상생활에서 흔히 겪는 증상입니다. 만약 두통이 반복적으로 발생한다면,...",두통은 우리 일상생활에서 흔히 겪는 증상입니다. 두통의 치료는 그 원인과 심각성에 ...
5514,응급질환,두통,치료,두통은 많은 사람들이 겪는 일상적인 문제입니다. 치료 방법은 두통의 종류와 원인에 ...,두통은 많은 사람들이 겪는 일상적인 문제입니다. 치료 방법은 두통의 종류와 원인에 ...
5515,응급질환,두통,치료,두통은 일상 생활에 불편을 초래할 수 있는 주요한 증상 중 하나입니다. 치료법은 두...,두통은 일상 생활에 불편을 초래할 수 있는 주요한 증상 중 하나입니다. 치료법은 두...
5516,응급질환,두통,치료,두통은 일반적으로 일차성 두통과 이차성 두통으로 구분됩니다. 일차성 두통은 두통을 ...,두통은 일반적으로 일차성 두통과 이차성 두통으로 구분됩니다. 일차성 두통은 두통을 ...


In [28]:
answer_list = answer_df['answer'].tolist()
intention_list = answer_df['intention'].tolist()

In [29]:
test_list = []
a='하이요'
[a]*2

['하이요', '하이요']

In [30]:
test_list.extend([a]*2)
test_list

['하이요', '하이요']

In [ ]:
final_chunk = []
answer_map = []
intention_map = []

for answer in answer_list:
    # 2문장씩 나누기
    sentence_chunk = split_sentences(answer, max_sentences=2)

    # 긴 문장 쪼개기
    for chunk in sentence_chunk:
        split_chunks = split_text_with_recursive_splitter(chunk, chunk_size=300, chunk_overlap=50)
        final_chunk.extend(split_chunks)

        # 분할된 chunk마다 원본 맵핑
        answer_map.extend([answer] * len(split_chunks))
        # 분할된 chunk마다 답변 의도 맵핑
        intention = answer_df[answer_df['answer'] == answer]['intention'].values[0] # series 형태여서 values로 텍스트만 가져옴.
        intention_map.extend([intention] * len(split_chunks))


In [52]:
len(final_chunk), final_chunk[:10]

(23635,
 ['두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 자세한 관리는 두통 예방에 도움이 될 수 있습니다.',
  '컴퓨터를 사용할 때는 모니터를 눈높이에 배치하는 것이 좋습니다. 의자에 앉을 때는 등을 펴고 허리를 곧게 세워 앉습니다.',
  '책을 읽을 때는 눈높이에 맞추어 읽으며 책을 읽을 때는 한손으로 받친 자세로 읽는 것이 좋습니다. 모니터를 멀리 배치할 경우 눈의 피로를 줄일 수 있습니다.',
  '긴 시간동안 앉아 있는 경우에는 일어나서 스트레칭을 해주거나 몸을 좌우로 비틀어서 자세를 풀어주는 것이 좋습니다. 또한, 일상생활에서 적절한 자세를 유지하는 것과 충분한 수면을 취함으로써 두통 예방에 도움을 줄 수 있습니다.',
  '올바른 자세와 수면, 스트레칭을 통해 일상 생활에서 올바른 자세를 유지하는 것이 두통 예방에 도움이 됩니다.',
  '긴장성 두통을 예방하기 위한 몇 가지 조치를 취할 수 있습니다. 먼저, 건강한 생활 습관을 유지하는 것이 중요합니다.',
  '규칙적인 운동과 올바른 식단은 혈압을 조절하고 신경성 두통을 예방하는 데 도움이 됩니다. 또한, 스트레스 관리도 중요합니다.',
  '스트레스는 긴장성 두통의 주된 원인이므로, 적절한 휴식과 명상을 통해 스트레스를 효과적으로 관리해야 합니다. 마지막으로, 의사나 영양사와의 상담을 통해 적절한 예방 조치를 취하는 것도 중요합니다.',
  '의사나 영양사의 조언을 따르고 정기적인 건강 관리와 스트레스 관리를 하면 긴장성 두통을 예방하는 데 도움이 될 것입니다.',
  '두통 예방을 위한 생활습관은 다양한 방법으로 개선할 수 있습니다. 일상적인 습관 중 하나는 충분한 수면을 취하는 것입니다.'])

In [51]:
len(answer_map), answer_map[:10]

(23635,
 ['두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 자세한 관리는 두통 예방에 도움이 될 수 있습니다. 컴퓨터를 사용할 때는 모니터를 눈높이에 배치하는 것이 좋습니다. 의자에 앉을 때는 등을 펴고 허리를 곧게 세워 앉습니다. 책을 읽을 때는 눈높이에 맞추어 읽으며 책을 읽을 때는 한손으로 받친 자세로 읽는 것이 좋습니다. 모니터를 멀리 배치할 경우 눈의 피로를 줄일 수 있습니다. 긴 시간동안 앉아 있는 경우에는 일어나서 스트레칭을 해주거나 몸을 좌우로 비틀어서 자세를 풀어주는 것이 좋습니다. 또한, 일상생활에서 적절한 자세를 유지하는 것과 충분한 수면을 취함으로써 두통 예방에 도움을 줄 수 있습니다. 올바른 자세와 수면, 스트레칭을 통해 일상 생활에서 올바른 자세를 유지하는 것이 두통 예방에 도움이 됩니다.',
  '두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 자세한 관리는 두통 예방에 도움이 될 수 있습니다. 컴퓨터를 사용할 때는 모니터를 눈높이에 배치하는 것이 좋습니다. 의자에 앉을 때는 등을 펴고 허리를 곧게 세워 앉습니다. 책을 읽을 때는 눈높이에 맞추어 읽으며 책을 읽을 때는 한손으로 받친 자세로 읽는 것이 좋습니다. 모니터를 멀리 배치할 경우 눈의 피로를 줄일 수 있습니다. 긴 시간동안 앉아 있는 경우에는 일어나서 스트레칭을 해주거나 몸을 좌우로 비틀어서 자세를 풀어주는 것이 좋습니다. 또한, 일상생활에서 적절한 자세를 유지하는 것과 충분한 수면을 취함으로써 두통 예방에 도움을 줄 수 있습니다. 올바른 자세와 수면, 스트레칭을 통해 일상 생활에서 올바른 자세를 유지하는 것이 두통 예방에 도움이 됩니다.',
  '두통을 예방하기 위해서는 일상적으로 바른 자세를 유지해야 합니다. 일상 생활에서의 자세한 관리는 두통 예방에 도움이 될 수 있습니다. 컴퓨터를 사용할 때는 모니터를 눈높이에 배치하는 것이 좋습니다. 의자에 앉을 때는 등을 펴고 허리를 곧게 세워 앉습니다. 

In [50]:
len(intention_map), intention_map[:10]

(23635, ['예방', '예방', '예방', '예방', '예방', '예방', '예방', '예방', '예방', '예방'])

#### 2) 답변 임베딩 및 답변 벡터 DB 만들기

In [ ]:
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
embeddings = model.encode(final_chunk)
print('생성된 임베딩 개수: ', len(embeddings))

생성된 임베딩 개수:  23635


In [38]:
embeddings.shape

(23635, 768)

- 답변 벡터 DB 생성

In [42]:
import numpy as np

# faiss 인덱스 생성 및 데이터 삽입
dimension = embeddings.shape[1] # 임베딩 벡터의 차원
index = faiss.IndexFlatL2(dimension)  # L2 거리(유클리드 거리) 기반으로 검색을 수행하도록 설정(인덱스 생성). 벡터 검색 기계를 만듬.
index.add(np.array(embeddings)) # 각 벡터를 L2 거리 기반 인덱스에 추가하고, 검색을 빠르게 할 수 있도록 데이터 구조를 최적화

# 디버깅용 출력: 저장된 벡터 개수 확인
print(f"FAISS 인덱스에 저장된 벡터 개수: {index.ntotal}")

FAISS 인덱스에 저장된 벡터 개수: 23635


## 3. 벡터 DB에서 검색
1) 질문을 받고 의도를 예측한다 (학습시킨 분류 모델 사용)
2) 의도에 해당하는 답변 DB만 추출
3) 임베딩한 질문과 가장 유사한 답변을 추출한 답변 벡터 DB에서 찾는다(faiss 사용)

- 1) 질문 의도 예측 

In [43]:
question_df = pd.read_csv('두통 질문.csv', index_col=0)
question_df

,disease_category,disease_name,intention,question
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?
...,...,...,...,...
3423,응급질환,두통,치료,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...
3424,응급질환,두통,치료,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.
3425,응급질환,두통,치료,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.
3426,응급질환,두통,치료,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?


In [44]:
question = '오른쪽 머리에서 찌릿하는 두통은 뭔가요?'
question_vec = model.encode([question], batch_size=32)

In [45]:
question_ss = scaler.transform(question_vec)
question_pca = pca.transform(question_ss)
result = rf_model.predict(question_pca)
result

array([3], dtype=int64)

In [46]:
label = le.inverse_transform([result])
label[0]

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


'증상'

- 2) 의도에 해당하는 답변 DB만 추출

In [47]:
# 분류 결과와 같은 증상의 인덱스만 추출
filtered_intention_idx = []
for idx, text in enumerate(intention_map):
    if text == label[0]:
        filtered_intention_idx.append(idx)

len(filtered_intention_idx), filtered_intention_idx

(1335,
 [16392,
  16393,
  16394,
  16395,
  16396,
  16397,
  16398,
  16399,
  16400,
  16401,
  16402,
  16403,
  16404,
  16405,
  16406,
  16407,
  16408,
  16409,
  16410,
  16411,
  16412,
  16413,
  16414,
  16415,
  16416,
  16417,
  16418,
  16419,
  16420,
  16421,
  16422,
  16423,
  16424,
  16425,
  16426,
  16427,
  16428,
  16429,
  16430,
  16431,
  16432,
  16433,
  16434,
  16435,
  16436,
  16437,
  16438,
  16439,
  16440,
  16441,
  16442,
  16443,
  16444,
  16445,
  16446,
  16447,
  16448,
  16449,
  16450,
  16451,
  16452,
  16453,
  16454,
  16455,
  16456,
  16457,
  16458,
  16459,
  16460,
  16461,
  16462,
  16463,
  16464,
  16465,
  16466,
  16467,
  16468,
  16469,
  16470,
  16471,
  16472,
  16473,
  16474,
  16475,
  16476,
  16477,
  16478,
  16479,
  16480,
  16481,
  16482,
  16483,
  16484,
  16485,
  16486,
  16487,
  16488,
  16489,
  16490,
  16491,
  16492,
  16493,
  16494,
  16495,
  16496,
  16497,
  16498,
  16499,
  16500,
  16501,
  1

In [59]:
# intention이 증상인지 확인
answer_df[['intention', 'answer']].loc[answer_df['answer']==answer_map[16392]]

,intention,answer
3811,증상,"두통은 흔히 뇌 혈액 순환 장애로 인해 발생하며, 두통은 일상 생활에 직접적인 영향..."


- 3. 임베딩한 질문과 가장 유사한 답변을 답변 벡터 DB에서 찾음

In [61]:
a = np.array(['a', 'b', 'c', 'd', 'e'])
li = [0, 2]
a[li]

array(['a', 'c'], dtype='<U1')

In [62]:
# 필터링된 인덱스에 해당하는 부분만 벡터 DB로 재생성
filtered_index = faiss.IndexFlatL2(dimension) # 차원 설정
filtered_index.add(np.array(embeddings)[filtered_intention_idx]) # np.array는 리스트를 리스트로 감싸서 필터링 가능

In [63]:
# 생성된 벡터 DB 개수
filtered_index.ntotal

1335

In [ ]:
fi_distances, fi_indicies = filtered_index.search(question_vec, 5)  # 쿼리 벡터와 가장 유사한(거리가 가까운) 벡터 검색

In [65]:
fi_distances, fi_indicies

(array([[164.31354, 173.27637, 175.1254 , 175.27673, 176.04108]],
       dtype=float32),
 array([[ 930,  581,  936, 1237,  805]], dtype=int64))

- 새로운 벡터 DB에서 나온 인덱스는 새로운 DB 기준의 인덱스 이므로 원래 벡터 DB에 해당하는 인덱스로 변환해야함.

In [66]:
fi_indicies[0]

array([ 930,  581,  936, 1237,  805], dtype=int64)

In [67]:
filtered_top_idx = [filtered_intention_idx[i] for i in fi_indicies[0]]
filtered_top_idx

[17322, 16973, 17328, 17629, 17197]

In [106]:
for j, i in enumerate(filtered_top_idx):
    # print(j, i)
    print(f'벡터 거리: {fi_distances[0][j]:.2f} \n답변: {answer_map[i]}')

벡터 거리: 164.31 
답변: 편두통은 일반적인 두통과 비슷한 증상을 보이는 경향이 있습니다. 그러나 편두통의 두통이 주로 한쪽 머리에만 나타나는 것과는 다른 점이 있습니다. 편두통은 일반적인 두통과는 다른 양상을 보입니다. 주로 한쪽 머리에 심한 통증을 일으키는 편두통은 일반적으로 한쪽 얼굴에 두통이 나타납니다. 또한 편두통은 구토, 빛에 대한 증상, 수면 장애, 어지러움과 같은 다양한 증상도 동반할 수 있습니다. 두통은 발작적으로 나타나며, 주로 밝은 불빛에 대한 민감도가 증가합니다. 이러한 특징을 통해 편두통은 주로 편두통 발작 시에 머리 통증뿐만 아니라 두통, 눈의 통증, 구토, 빛에 대한 민감도 증가 등의 다양한 증상도 동반할 수 있음을 알 수 있습니다.
벡터 거리: 173.28 
답변: 두통은 흔히 발생하는 증상으로, 일반적으로 통증의 크기나 지속시간이 다를 수 있습니다. 두통의 주요 증상은 한쪽에서 시작되는 두통입니다. 가장 특징적인 증상은 다음 날까지 계속되는 통증입니다. 발작적인 두통은 대부분 아침에 발생하며, 활동한 후에 완화됩니다. 두통은 머리와 턱을 포함한 머리 전체에서 발생하며, 일반적으로 욱신욱신한 아픔을 동반합니다. 두통은 일시적인 편두통과 만성적인 두통으로 나뉩니다. 편두통은 매우 심한 두통으로, 두통 발작 사이에 통증의 강도는 약해져 견딜 수 있을 정도가 됩니다. 또한 통증이 한 쪽이나 양쪽 귀에서 느껴지거나, 다른 쪽 귀에서 느껴질 수도 있습니다. 두통은 여러 요인에 의해 발생할 수 있으며, 이러한 이유로 정확한 원인을 파악하기 위해 진단 검사가 필요합니다.
벡터 거리: 175.13 
답변: 두통은 흔한 증상이며, 일반적으로 한쪽 머리에서 시작하여 욱신욱신한 느낌이나 통증이 발생합니다. 심한 두통은 잠을 자거나 활동한 후에 완화되는 경향이 있습니다. 통증은 대부분의 경우 편측성을 띠며, 가끔은 양쪽에서도 발생할 수 있습니다. 두통 발작은 주로 아침에 증상이 나타나고, 오후에는 완화됩니다. 두통은 한쪽 편에만 발생하는 경우도 

In [ ]:
top3_answers = []
for j, i in enumerate(filtered_top_idx):
    top3_answers.append({f'연관 답변{j+1}: {answer_map[i]}'})

top3_answers

[{'연관 답변1: 편두통은 일반적인 두통과 비슷한 증상을 보이는 경향이 있습니다. 그러나 편두통의 두통이 주로 한쪽 머리에만 나타나는 것과는 다른 점이 있습니다. 편두통은 일반적인 두통과는 다른 양상을 보입니다. 주로 한쪽 머리에 심한 통증을 일으키는 편두통은 일반적으로 한쪽 얼굴에 두통이 나타납니다. 또한 편두통은 구토, 빛에 대한 증상, 수면 장애, 어지러움과 같은 다양한 증상도 동반할 수 있습니다. 두통은 발작적으로 나타나며, 주로 밝은 불빛에 대한 민감도가 증가합니다. 이러한 특징을 통해 편두통은 주로 편두통 발작 시에 머리 통증뿐만 아니라 두통, 눈의 통증, 구토, 빛에 대한 민감도 증가 등의 다양한 증상도 동반할 수 있음을 알 수 있습니다.'},
 {'연관 답변2: 두통은 흔히 발생하는 증상으로, 일반적으로 통증의 크기나 지속시간이 다를 수 있습니다. 두통의 주요 증상은 한쪽에서 시작되는 두통입니다. 가장 특징적인 증상은 다음 날까지 계속되는 통증입니다. 발작적인 두통은 대부분 아침에 발생하며, 활동한 후에 완화됩니다. 두통은 머리와 턱을 포함한 머리 전체에서 발생하며, 일반적으로 욱신욱신한 아픔을 동반합니다. 두통은 일시적인 편두통과 만성적인 두통으로 나뉩니다. 편두통은 매우 심한 두통으로, 두통 발작 사이에 통증의 강도는 약해져 견딜 수 있을 정도가 됩니다. 또한 통증이 한 쪽이나 양쪽 귀에서 느껴지거나, 다른 쪽 귀에서 느껴질 수도 있습니다. 두통은 여러 요인에 의해 발생할 수 있으며, 이러한 이유로 정확한 원인을 파악하기 위해 진단 검사가 필요합니다.'},
 {'연관 답변3: 두통은 흔한 증상이며, 일반적으로 한쪽 머리에서 시작하여 욱신욱신한 느낌이나 통증이 발생합니다. 심한 두통은 잠을 자거나 활동한 후에 완화되는 경향이 있습니다. 통증은 대부분의 경우 편측성을 띠며, 가끔은 양쪽에서도 발생할 수 있습니다. 두통 발작은 주로 아침에 증상이 나타나고, 오후에는 완화됩니다. 두통은 한쪽 편에만 발생하는 경우도 있고 양쪽에 나타나는 경우도 있습

## 4. fine tunning한 LLM 모델에 최종 사용하기 위해 클래스로 만들기

In [1]:
import faiss
import re
import numpy as np
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


class SelectReferenceAnswer:

    def __init__(self, answer_df, scaler, pca, rf_model, embed_model, labelencoder):
        self.df = answer_df
        self.scaler = scaler
        self.pca = pca
        self.rfmodel = rf_model
        self.embed_model = embed_model
        self.le = labelencoder
        
        # 변수 초기화
        self.embeddings = None
        self.dimension = None

    # 벡터 DB 만드는 메서드
    def make_vec_db(self):

        # 2문장씩 문장을 나누는 함수
        def split_sentences(sentences, max_sentences=2):
            sentence_list = re.split(r'(?<=[.!?]) +', sentences) # 문장을 마침표, 느낌표, 물음표 뒤에오는 공백을 기준으로 나눔(공백 뒤에 +를 붙여서 하나이상을 의미)
            chunks = []
            for i in range(0, len(sentence_list), max_sentences): # max_sentences 간격만큼
                chunks.append(' '.join(sentence_list[i:i + max_sentences])) # join으로 두개의 문장 공백으로 연결, 리스트 대신 문자열로 생성됨.
            
            return chunks

        # chunks 속 2문장씩 결합되어있는 문장들의 길이가 너무 긴 경우 나눠주기 위함.
        def split_text_with_recursive_splitter(text, chunk_size=300, chunk_overlap=50):
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size = chunk_size, # 최대 몇자로 설정할지
                chunk_overlap = chunk_overlap # 50개 겹쳐서 시작
            )

            return text_splitter.split_text(text)


        answer_list = self.df['answer'].tolist()
        intention_list = self.df['intention'].tolist()
        le_intention = self.le.fit_transform(self.df['intention'])


        final_chunk = []
        answer_map = []
        intention_map = []

        for answer in answer_list:
            # 2문장씩 나누기
            sentence_chunk = split_sentences(answer, max_sentences=2)

            # 긴 문장 쪼개기
            for chunk in sentence_chunk:
                split_chunks = split_text_with_recursive_splitter(chunk, chunk_size=300, chunk_overlap=50)
                final_chunk.extend(split_chunks)

                # 분할된 chunk마다 원본 맵핑
                answer_map.extend([answer] * len(split_chunks))
                # 분할된 chunk마다 질문 의도 맵핑
                intention = self.df[self.df['answer'] == answer]['intention'].values[0] # series 형태여서 values로 텍스트만 가져옴.
                intention_map.extend([intention] * len(split_chunks))

        # 임베딩
        self.embeddings = self.embed_model.encode(final_chunk)
        
        self.dimension = self.embeddings.shape[1] # 임베딩 벡터의 차원
        self.index = faiss.IndexFlatL2(self.dimension)  # L2 거리(유클리드 거리) 기반으로 검색을 수행하도록 설정(인덱스 생성)
        self.index.add(np.array(self.embeddings)) # 각 벡터를 L2 거리 기반 인덱스에 추가하고, 검색을 빠르게 할 수 있도록 데이터 구조를 최적화

        return intention_map, answer_map

    def search_answer(self, question, intention_map, answer_map, answer_num=3):
        
        #질문 임베딩
        question_vec = self.embed_model.encode([question], batch_size=32)

        question_ss = self.scaler.transform(question_vec)
        question_pca = self.pca.transform(question_ss)
        result = self.rfmodel.predict(question_pca)

        label = self.le.inverse_transform([result])

        # 분류 결과와 같은 증상의 인덱스만 추출
        filtered_intention_idx = []
        for idx, text in enumerate(intention_map):
            if text == label[0]:
                filtered_intention_idx.append(idx)

        # 필터링된 인덱스에 해당하는 부분만 벡터 DB로 재생성
        filtered_index = faiss.IndexFlatL2(self.dimension) # 차원 설정
        filtered_index.add(np.array(self.embeddings)[filtered_intention_idx]) # np.array는 리스트를 리스트로 감싸서 필터링 가능

        fi_distances, fi_indicies = filtered_index.search(question_vec, answer_num)  # 쿼리 벡터와 가장 유사한(거리가 가까운) 벡터 검색
        filtered_top_idx = [filtered_intention_idx[i] for i in fi_indicies[0]]
        
        top_answers = []
        for j, i in enumerate(filtered_top_idx):
            top3_answers.append({f'참고 정보{j+1}: {answer_map[i]}'})
        
        return top_answers

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


- 생성한 Class를 .py 형식으로 저장

In [2]:
with open('SearchAnswers.py', 'w') as f:
    f.write('''
import faiss
import re
import numpy as np
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


class SelectReferenceAnswer:

    def __init__(self, answer_df, scaler, pca, rf_model, embed_model, labelencoder):
        self.df = answer_df
        self.scaler = scaler
        self.pca = pca
        self.rfmodel = rf_model
        self.embed_model = embed_model
        self.le = labelencoder
        
        # 변수 초기화
        self.embeddings = None
        self.dimension = None

    # 벡터 DB 만드는 메서드
    def make_vec_db(self):

        # 2문장씩 문장을 나누는 함수
        def split_sentences(sentences, max_sentences=2):
            sentence_list = re.split(r'(?<=[.!?]) +', sentences) # 문장을 마침표, 느낌표, 물음표 뒤에오는 공백을 기준으로 나눔(공백 뒤에 +를 붙여서 하나이상을 의미)
            chunks = []
            for i in range(0, len(sentence_list), max_sentences): # max_sentences 간격만큼
                chunks.append(' '.join(sentence_list[i:i + max_sentences])) # join으로 두개의 문장 공백으로 연결, 리스트 대신 문자열로 생성됨.
            
            return chunks

        # chunks 속 2문장씩 결합되어있는 문장들의 길이가 너무 긴 경우 나눠주기 위함.
        def split_text_with_recursive_splitter(text, chunk_size=300, chunk_overlap=50):
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size = chunk_size, # 최대 몇자로 설정할지
                chunk_overlap = chunk_overlap # 50개 겹쳐서 시작
            )

            return text_splitter.split_text(text)


        answer_list = self.df['answer'].tolist()
        intention_list = self.df['intention'].tolist()
        le_intention = self.le.fit_transform(self.df['intention'])


        final_chunk = []
        answer_map = []
        intention_map = []

        for answer in answer_list:
            # 2문장씩 나누기
            sentence_chunk = split_sentences(answer, max_sentences=2)

            # 긴 문장 쪼개기
            for chunk in sentence_chunk:
                split_chunks = split_text_with_recursive_splitter(chunk, chunk_size=300, chunk_overlap=50)
                final_chunk.extend(split_chunks)

                # 분할된 chunk마다 원본 맵핑
                answer_map.extend([answer] * len(split_chunks))
                # 분할된 chunk마다 질문 의도 맵핑
                intention = self.df[self.df['answer'] == answer]['intention'].values[0] # series 형태여서 values로 텍스트만 가져옴.
                intention_map.extend([intention] * len(split_chunks))

        # 임베딩
        self.embeddings = self.embed_model.encode(final_chunk)
        
        self.dimension = self.embeddings.shape[1] # 임베딩 벡터의 차원
        self.index = faiss.IndexFlatL2(self.dimension)  # L2 거리(유클리드 거리) 기반으로 검색을 수행하도록 설정(인덱스 생성)
        self.index.add(np.array(self.embeddings)) # 각 벡터를 L2 거리 기반 인덱스에 추가하고, 검색을 빠르게 할 수 있도록 데이터 구조를 최적화

        return intention_map, answer_map

    def search_answer(self, question, intention_map, answer_map, answer_num=3):
        
        #질문 임베딩
        question_vec = self.embed_model.encode([question], batch_size=32)

        question_ss = self.scaler.transform(question_vec)
        question_pca = self.pca.transform(question_ss)
        result = self.rfmodel.predict(question_pca)

        label = self.le.inverse_transform([result])

        # 분류 결과와 같은 증상의 인덱스만 추출
        filtered_intention_idx = []
        for idx, text in enumerate(intention_map):
            if text == label[0]:
                filtered_intention_idx.append(idx)

        # 필터링된 인덱스에 해당하는 부분만 벡터 DB로 재생성
        filtered_index = faiss.IndexFlatL2(self.dimension) # 차원 설정
        filtered_index.add(np.array(self.embeddings)[filtered_intention_idx]) # np.array는 리스트를 리스트로 감싸서 필터링 가능

        fi_distances, fi_indicies = filtered_index.search(question_vec, answer_num)  # 쿼리 벡터와 가장 유사한(거리가 가까운) 벡터 검색
        filtered_top_idx = [filtered_intention_idx[i] for i in fi_indicies[0]]
        
        top_answers = []
        for j, i in enumerate(filtered_top_idx):
            top_answers.append({f'참고 정보{j+1}: {answer_map[i]}'})
        
        return top_answers
    ''')

In [4]:
import faiss
import numpy as np
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import joblib
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
from SearchAnswers import SelectReferenceAnswer

# 저장된 RandomForestClassifier 모델 불러오기
rf_model = joblib.load('./headache_randomforest.pkl')
scaler = joblib.load('./headache_scaler.pkl')
pca = joblib.load('./headache_pca.pkl')

answer_df = pd.read_csv("두통 답변.csv", index_col=0)
embed_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
labelencoder = LabelEncoder()

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from versio

In [5]:
# 객체 생성
select_answers = SelectReferenceAnswer(answer_df, scaler, pca, rf_model, embed_model, labelencoder)

In [6]:
# 벡터 DB 만들기
intention_map, answer_map = select_answers.make_vec_db()

In [9]:
question = '오른쪽 머리에서 찌릿하는 두통은 뭔가요?'
result = select_answers.search_answer(question, intention_map, answer_map)
result

c:\Users\Paris\miniconda3\envs\ds_study\Lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[{'참고 정보1: 편두통은 일반적인 두통과 비슷한 증상을 보이는 경향이 있습니다. 그러나 편두통의 두통이 주로 한쪽 머리에만 나타나는 것과는 다른 점이 있습니다. 편두통은 일반적인 두통과는 다른 양상을 보입니다. 주로 한쪽 머리에 심한 통증을 일으키는 편두통은 일반적으로 한쪽 얼굴에 두통이 나타납니다. 또한 편두통은 구토, 빛에 대한 증상, 수면 장애, 어지러움과 같은 다양한 증상도 동반할 수 있습니다. 두통은 발작적으로 나타나며, 주로 밝은 불빛에 대한 민감도가 증가합니다. 이러한 특징을 통해 편두통은 주로 편두통 발작 시에 머리 통증뿐만 아니라 두통, 눈의 통증, 구토, 빛에 대한 민감도 증가 등의 다양한 증상도 동반할 수 있음을 알 수 있습니다.'},
 {'참고 정보2: 두통은 흔히 발생하는 증상으로, 일반적으로 통증의 크기나 지속시간이 다를 수 있습니다. 두통의 주요 증상은 한쪽에서 시작되는 두통입니다. 가장 특징적인 증상은 다음 날까지 계속되는 통증입니다. 발작적인 두통은 대부분 아침에 발생하며, 활동한 후에 완화됩니다. 두통은 머리와 턱을 포함한 머리 전체에서 발생하며, 일반적으로 욱신욱신한 아픔을 동반합니다. 두통은 일시적인 편두통과 만성적인 두통으로 나뉩니다. 편두통은 매우 심한 두통으로, 두통 발작 사이에 통증의 강도는 약해져 견딜 수 있을 정도가 됩니다. 또한 통증이 한 쪽이나 양쪽 귀에서 느껴지거나, 다른 쪽 귀에서 느껴질 수도 있습니다. 두통은 여러 요인에 의해 발생할 수 있으며, 이러한 이유로 정확한 원인을 파악하기 위해 진단 검사가 필요합니다.'},
 {'참고 정보3: 두통은 흔한 증상이며, 일반적으로 한쪽 머리에서 시작하여 욱신욱신한 느낌이나 통증이 발생합니다. 심한 두통은 잠을 자거나 활동한 후에 완화되는 경향이 있습니다. 통증은 대부분의 경우 편측성을 띠며, 가끔은 양쪽에서도 발생할 수 있습니다. 두통 발작은 주로 아침에 증상이 나타나고, 오후에는 완화됩니다. 두통은 한쪽 편에만 발생하는 경우도 있고 양쪽에 나타나는 경우도 있습